In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession, Window

In [0]:
spark = SparkSession.builder.appName("someProblem").getOrCreate()

data = [
 (1, "2024-08-15"),
 (1, "2024-03-10"),
 (1, "2023-05-05"),
 (2, "2025-02-24"),
 (2, "2024-07-15"),
 (2, "2024-03-15")
]

# Define schema
schema = T.StructType([
    T.StructField("id", T.IntegerType(), True),
    T.StructField("plan_date", T.StringType(), True)
])

# Create DataFrame
df = spark.createDataFrame(data, schema=schema)

In [0]:
# Create a window partitioned by 'id' and ordered by 'plan_date'
six_month_window = Window.partitionBy("id").orderBy(F.desc("plan_date"))

# Calculate the difference between the current and previous 'plan_date' in days
df = df.withColumn("month_lag", F.lag("plan_date").over(six_month_window)).\
withColumn(
    "is_six_month", 
    F.when(
        (F.months_between(F.col("month_lag"), F.col("plan_date")) < 6) | (F.col("month_lag").isNull()), 
        F.lit("YES")
    ).otherwise(F.lit("NO"))
).\
    withColumn("is_current_fiscal_year", F.when(F.col("plan_date").between("2024-04-01", 
                                                                   "2025-03-31"), "YES").otherwise("NO")).\
    withColumn("is_current_year", F.when(F.col("plan_date").between("2024-01-01", 
                                                                   "2024-12-31"), "YES").otherwise("NO")).\
    drop("month_lag").show()



+---+----------+------------+----------------------+---------------+
| id| plan_date|is_six_month|is_current_fiscal_year|is_current_year|
+---+----------+------------+----------------------+---------------+
|  1|2024-08-15|         YES|                   YES|            YES|
|  1|2024-03-10|         YES|                    NO|            YES|
|  1|2023-05-05|          NO|                    NO|             NO|
|  2|2025-02-24|         YES|                   YES|             NO|
|  2|2024-07-15|          NO|                   YES|            YES|
|  2|2024-03-15|         YES|                    NO|            YES|
+---+----------+------------+----------------------+---------------+

